In [83]:
import pandas as pd
import numpy as np
import re
import datetime

In [84]:
def convert_to_euro(value):
    numeric_part = re.findall(r'\d+\.?\d*', value)[0]
    unit = value[-1]
    if unit == 'm':
        return float(numeric_part) * 1000000
    elif unit == 'k':
        return float(numeric_part) * 1000
    else:
        return None

def convert_to_euro1(value):
    try:
        value = str(value)
        numeric_value = value[1:-1]
        if value.endswith('m'):
            multiplier = 1000000
        elif value.endswith('k'):
            multiplier = 1000
        else:
            raise ValueError("Invalid value format")
        euro_value = float(numeric_value) * multiplier
        return euro_value
    except:
        return None

def extract_number(url):
    try:
        parts = str(url).split('/')
        number = parts[-1].strip()
        return int(number)
    except:
        return None

def convert_to_year(value):
    try:
        ex_year = int(value[:2])
        if ex_year<= 25:
            year = ex_year + 2000
        else:
            year = ex_year + 1900
        return int(year)
    except:
        return None
    
def extract_number2(url):
    number = re.search(r'\d+', url)
    return int(number.group()) if number else None

def clean_int(value):
    try:
        return float(value)
    except:
        return 0

In [85]:
national = pd.read_excel('Mentor_Data.xlsx',sheet_name='national')
national['id'] = national['country_url'].str.extract(r'wettbewerbe/(\d+)/saison_id')
national = national.drop('country_url', axis=1)
national = national.drop_duplicates()
national = national.reset_index(drop=True)
national['id'] = national['id'].astype(int)
national

,id,name
0,50,France
1,157,Spain
2,75,Italy
3,189,England
4,40,Germany


In [86]:
competitions = pd.read_excel('Mentor_Data.xlsx',sheet_name='competitions')
competitions['id'] = competitions['league_cups_url'].apply(lambda x: x.split('www.transfermarkt.com/')[1].split('/')[0])
competitions['country_id'] = competitions['country_url'].str.extract(r'wettbewerbe/(\d+)/saison_id')
competitions = competitions.drop(['league_cups_url','country_url'], axis=1)
competitions = competitions.drop_duplicates(subset='id')
competitions = competitions.reset_index(drop=True)
competitions['country_id'] = competitions['country_id'].astype(int)
competitions

,id,name,teams_number,market_value,players_number,avg_age,foreigners,country_id
0,ligue-1,Ligue 1,NaN,NaN,NaN,NaN,NaN,50
1,ligue-2,Ligue 2,NaN,NaN,NaN,NaN,NaN,50
2,championnat-national,Championnat National,NaN,NaN,NaN,NaN,NaN,50
3,championnat-national-2-groupe-a,CFA - Grp. A,NaN,NaN,NaN,NaN,NaN,50
4,championnat-national-2-groupe-b,CFA - Grp. B,NaN,NaN,NaN,NaN,NaN,50
...,...,...,...,...,...,...,...,...
258,verbandsliga-schleswig-holstein-nord-ost,Verbandsliga SH Nord-Ost,NaN,NaN,NaN,NaN,NaN,40
259,verbandsliga-schleswig-holstein-nord-west-beta-,Verbandsliga SH Nord-West,NaN,NaN,NaN,NaN,NaN,40
260,verbandsliga-schleswig-holstein-sud-west-beta-,Verbandsliga SH Süd-West,NaN,NaN,NaN,NaN,NaN,40
261,aufstiegsrunde-zur-landesliga-schleswig-holstein,Aufstiegsr. Landesliga,NaN,NaN,NaN,NaN,NaN,40


In [87]:
clubs = pd.read_excel('Mentor_Data.xlsx',sheet_name='clubs')
clubs['id'] = clubs['team_url'].apply(lambda x: x.split('/verein/')[1].split('/')[0])
clubs['league_id'] = clubs['league_url'].apply(lambda x: x.split('www.transfermarkt.com/')[1].split('/')[0])
clubs = clubs.drop(['team_url','league_url'], axis=1)
clubs['value'] = clubs['value'].apply(convert_to_euro)
clubs = clubs.drop_duplicates(subset='id')
clubs = clubs.reset_index(drop=True)
clubs['id'] = clubs['id'].astype(int)
clubs

,id,name,country_id,foundation_date,value,stadium,squad_size,average_age,national_players,current_transfer_record,foreigners,season,league_id
0,631,Chelsea FC,NaN,NaN,13840000.0,NaN,40,24.7,NaN,NaN,31,2015,premier-league
1,281,Manchester City,NaN,NaN,13140000.0,NaN,38,25.4,NaN,NaN,27,2015,premier-league
2,11,Arsenal FC,NaN,NaN,11170000.0,NaN,41,24.7,NaN,NaN,29,2015,premier-league
3,985,Manchester United,NaN,NaN,10000000.0,NaN,44,23.5,NaN,NaN,26,2015,premier-league
4,31,Liverpool FC,NaN,NaN,8530000.0,NaN,44,23.4,NaN,NaN,27,2015,premier-league
...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,3522,Spezia Calcio,NaN,NaN,2310000.0,NaN,34,23.6,NaN,NaN,23,2021,serie-a
143,380,US Salernitana 1919,NaN,NaN,1380000.0,NaN,51,25.9,NaN,NaN,26,2021,serie-a
144,4171,Benevento Calcio,NaN,NaN,1200000.0,NaN,48,25.1,NaN,NaN,21,2017,serie-a
145,19,Brescia Calcio,NaN,NaN,2520000.0,NaN,33,25.9,NaN,NaN,11,2019,serie-a


In [88]:
players = pd.read_excel('Mentor_Data.xlsx',sheet_name='players')
players = players.dropna(subset=['name'])
players['id'] = players['player_url'].apply(extract_number).astype(int)
players['current_club_id'] = players['current_club_url'].apply(extract_number)
players['current_value'] = players['current_value'].apply(convert_to_euro1)
players = players.drop(['player_url','current_club_url'], axis=1)
players['age'] = datetime.date.today().year - players['birth_date'].dt.year
players = players.drop_duplicates(subset='id')
players = players.reset_index(drop=True)
players['id'] = players['id'].astype(int)
players.replace({np.nan: None}, inplace=True)
#players['current_club_id'] = players['current_club_id'].astype(int)
players

,id,name,birth_date,age,height,current_club_id,club_joined,contract_expires,birth_place,citizenship,position,national_id,current_value,agent,foot,shirt_number
0,207302,Sergio Rico,1993-09-01 00:00:00,30.0,196.0,583.0,2022-01-21 00:00:00,None,None,None,Goalkeeper,None,4000000.0,None,right,None
1,316884,Dominik Greif,1997-04-06 00:00:00,26.0,197.0,237.0,2021-07-06 00:00:00,None,None,None,Goalkeeper,None,2000000.0,None,right,13.0
2,29051,Manolo Reina,1985-04-01 00:00:00,38.0,186.0,1084.0,2017-07-06 00:00:00,None,None,None,Goalkeeper,None,200000.0,None,right,None
3,632913,Pere García,2002-03-22 00:00:00,21.0,193.0,8494.0,None,None,None,None,Goalkeeper,None,None,None,right,None
4,741285,Ferran Quetglás,2005-06-06 00:00:00,18.0,188.0,28870.0,None,None,None,None,Goalkeeper,None,None,None,right,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8879,413459,Eris Abedini,1998-08-29 00:00:00,25.0,190.0,242.0,None,None,None,None,Defensive Midfield,None,300000.0,None,right,None
8880,284872,Álvaro Bravo,1998-02-04 00:00:00,25.0,181.0,16486.0,None,None,None,None,Central Midfield,None,200000.0,None,right,None
8881,696883,Juan Brunet,1998-01-24 00:00:00,25.0,190.0,16055.0,None,None,None,None,Central Midfield,None,50000.0,None,None,None
8882,636229,Carlos León,2002-12-05 00:00:00,21.0,185.0,515.0,None,None,None,None,Central Midfield,None,None,None,right,None


In [89]:
teams_season = pd.read_excel('Mentor_Data.xlsx',sheet_name='teams_season')
teams_season = teams_season.dropna(subset=['champion'])
teams_season['rank'] = 1
teams_season['competition_id'] = teams_season['league_cups_url'].apply(lambda x: x.split('www.transfermarkt.com/')[1].split('/')[0])
teams_season = teams_season.reset_index(drop=True)
teams_season['club_id'] = teams_season['champion']
teams_season = teams_season.drop(['league_cups_url', 'champion'], axis=1)
teams_season = teams_season.drop_duplicates()
teams_season = teams_season.reset_index(drop=True)
teams_season

,id,club_id,season,competition_id,rank,matches,win,draw,loss,goals_scored,goals_conceded,players_avg_age
0,NaN,Paris Saint-Germain,2015,ligue-1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,AS Nancy-Lorraine,2015,ligue-2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Paris Saint-Germain,2015,coupe-de-france,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Paris Saint-Germain,2015,trophee-des-champions,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Paris Saint-Germain,2015,coupe-de-la-ligue,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
536,NaN,VfL Osnabrück,2016,niedersachsenpokal-bis-17-18-,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
537,NaN,MSV Duisburg,2016,landespokal-niederrhein,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
538,NaN,Eintracht Norderstedt,2016,landespokal-hamburg,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
539,NaN,1.FC Magdeburg,2016,landespokal-sachsen-anhalt,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
players_season = pd.read_excel('Mentor_Data.xlsx',sheet_name='players_season')
players_season['player_id'] = players_season['player_url_details'].apply(lambda x: x.split('/spieler/')[1].split('/')[0]).astype(int)
players_season['club_id'] = players_season['club_url'].apply(lambda x: x.split('/verein/')[1].split('/')[0])
players_season['season'] = players_season['season'].apply(convert_to_year).replace(np.nan, None).astype(int)
players_season = players_season.drop(['club_url', 'player_url_details'], axis=1)
players_season = players_season.drop_duplicates()
players_season = players_season.reset_index(drop=True)
players_season['club_id'] = players_season['club_id'].astype(int)
players_season.iloc[:, 4:18] = players_season.iloc[:, 4:18].applymap(clean_int)
players_season['Height'] = players_season['Height'].str.replace(' m', '').str.replace(',', '.').astype(float) * 100
players_season

,id,player_id,season,club_id,squad,appearance,point_per_goal,goals,assits,own_goals,...,goals_conceded,clean_sheets,minutes_per_goal,minutes_played,competition,age,position,Height,Current international,Agent
0,NaN,305686,2022,1387,33.0,24.0,1.58,NaN,NaN,NaN,...,NaN,NaN,NaN,586.0,Serie C - B,25.0,Central Midfield,185.0,NaN,NaN
1,NaN,305686,2021,1253,1.0,0.0,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Playoff Serie C,25.0,Central Midfield,185.0,NaN,NaN
2,NaN,305686,2021,1253,25.0,22.0,1.41,1.0,NaN,NaN,...,NaN,NaN,809.0,809.0,Serie C-B,25.0,Central Midfield,185.0,NaN,NaN
3,NaN,305686,2020,4255,3.0,2.0,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,77.0,Serie C - B,25.0,Central Midfield,185.0,NaN,NaN
4,NaN,305686,2020,4255,2.0,2.0,1.50,NaN,NaN,NaN,...,NaN,NaN,NaN,99.0,Italy Cup,25.0,Central Midfield,185.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257082,NaN,5578,2000,595,2.0,2.0,1.50,NaN,NaN,NaN,...,2.0,NaN,NaN,180.0,Coupe de France,42.0,Goalkeeper,185.0,NaN,NaN
257083,NaN,5578,1999,595,20.0,3.0,0.33,NaN,NaN,NaN,...,5.0,NaN,NaN,270.0,Division 1,42.0,Goalkeeper,185.0,NaN,NaN
257084,NaN,5578,1999,595,1.0,0.0,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Coupe de la Ligue,42.0,Goalkeeper,185.0,NaN,NaN
257085,NaN,5578,1998,595,6.0,1.0,0.00,NaN,NaN,NaN,...,1.0,NaN,NaN,70.0,Division 1,42.0,Goalkeeper,185.0,NaN,NaN


In [91]:
transfers = pd.read_excel('Mentor_Data.xlsx',sheet_name='transfers')
transfers = transfers.dropna(subset=['season'])
transfers['signed_from_url'] = transfers['signed_from_url'].astype(str)
transfers['team_url'] = transfers['team_url'].astype(str)
mask = transfers['fee'].str.contains('€')
nan_mask = pd.isna(transfers['fee'])
valid_mask = mask & ~nan_mask
transfers.loc[~valid_mask, 'fee'] = np.nan
transfers['fee'] = transfers['fee'].apply(convert_to_euro1)
transfers['market_value'] = transfers['market_value'].apply(convert_to_euro1)
transfers['season'] = transfers['season'].dt.year
transfers['player_id'] = transfers['player_url'].apply(extract_number)
transfers['origin_club_id'] = transfers['signed_from_url'].apply(extract_number2)
transfers['destination_club_id'] = transfers['team_url'].apply(lambda x: x.split('/verein/')[1].split('/')[0])
transfers = transfers.drop_duplicates()
transfers = transfers.reset_index(drop=True)
transfers = transfers.drop(['player_url','signed_from_url','team_url'], axis=1)
transfers['player_id'] = transfers['player_id'].astype(int)
transfers['destination_club_id'] = transfers['destination_club_id'].astype(int)
transfers

,id,player_id,origin_club_id,destination_club_id,season,market_value,fee
0,NaN,207302,583.0,237,2022,4000000.0,NaN
1,NaN,316884,540.0,237,2021,2000000.0,2500000.0
2,NaN,2022,8494.0,237,2022,300000.0,NaN
3,NaN,29051,5648.0,237,2017,200000.0,NaN
4,NaN,277384,862.0,237,2019,4000000.0,1500000.0
...,...,...,...,...,...,...,...
22928,NaN,487688,418.0,16795,2020,3000000.0,2500000.0
22929,NaN,424784,1010.0,16795,2020,10000000.0,7000000.0
22930,NaN,610336,1084.0,16795,2020,1500000.0,1500000.0
22931,NaN,57825,3709.0,16795,2020,1500000.0,NaN


In [92]:
competitions_dict = competitions[['name','id']]
clubs_dict = clubs[['name','id']]

In [93]:
final_clubs_dict = {name: id for name, id in zip(clubs_dict['name'], clubs_dict['id'])}
final_comnpetitions_dict = {name: id for name, id in zip(competitions_dict['id'], competitions_dict['id'])}

teams_season = teams_season[teams_season['club_id'].isin(final_clubs_dict.keys())]
teams_season['club_id'] = teams_season['club_id'].replace(final_clubs_dict)

teams_season = teams_season[teams_season['competition_id'].isin(final_comnpetitions_dict.keys())]
#teams_season['competition_id'] = teams_season['competition_id'].replace(final_comnpetitions_dict)

teams_season = teams_season.reset_index(drop=True)
teams_season

C:\Users\battlebox\AppData\Local\Temp\ipykernel_6116\2919453595.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teams_season['club_id'] = teams_season['club_id'].replace(final_clubs_dict)


,id,club_id,season,competition_id,rank,matches,win,draw,loss,goals_scored,goals_conceded,players_avg_age
0,NaN,583,2015,ligue-1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1159,2015,ligue-2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,583,2015,coupe-de-france,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,583,2015,trophee-des-champions,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,583,2015,coupe-de-la-ligue,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
166,NaN,82,2015,dfl-supercup,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167,NaN,27,2016,bundesliga,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,NaN,79,2016,2-bundesliga,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,NaN,16,2016,dfb-pokal,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
players_season['competition'] = players_season['competition'].replace(final_comnpetitions_dict)
final_clubs_dict = {name: id for name, id in zip(clubs_dict['id'], clubs_dict['id'])}
players_season['club_id'] = players_season['club_id'].map(final_clubs_dict).fillna('')
final_comnpetitions_dict = {name: id for name, id in zip(competitions_dict['name'], competitions_dict['id'])}
players_season['competition'] = players_season['competition'].map(final_comnpetitions_dict).fillna('')
players_season

,id,player_id,season,club_id,squad,appearance,point_per_goal,goals,assits,own_goals,...,goals_conceded,clean_sheets,minutes_per_goal,minutes_played,competition,age,position,Height,Current international,Agent
0,NaN,305686,2022,,33.0,24.0,1.58,NaN,NaN,NaN,...,NaN,NaN,NaN,586.0,,25.0,Central Midfield,185.0,NaN,NaN
1,NaN,305686,2021,,1.0,0.0,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,play-off-serie-c,25.0,Central Midfield,185.0,NaN,NaN
2,NaN,305686,2021,,25.0,22.0,1.41,1.0,NaN,NaN,...,NaN,NaN,809.0,809.0,serie-c-girone-b,25.0,Central Midfield,185.0,NaN,NaN
3,NaN,305686,2020,,3.0,2.0,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,77.0,,25.0,Central Midfield,185.0,NaN,NaN
4,NaN,305686,2020,,2.0,2.0,1.50,NaN,NaN,NaN,...,NaN,NaN,NaN,99.0,coppa-italia,25.0,Central Midfield,185.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257082,NaN,5578,2000,595.0,2.0,2.0,1.50,NaN,NaN,NaN,...,2.0,NaN,NaN,180.0,coupe-de-france,42.0,Goalkeeper,185.0,NaN,NaN
257083,NaN,5578,1999,595.0,20.0,3.0,0.33,NaN,NaN,NaN,...,5.0,NaN,NaN,270.0,,42.0,Goalkeeper,185.0,NaN,NaN
257084,NaN,5578,1999,595.0,1.0,0.0,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,coupe-de-la-ligue,42.0,Goalkeeper,185.0,NaN,NaN
257085,NaN,5578,1998,595.0,6.0,1.0,0.00,NaN,NaN,NaN,...,1.0,NaN,NaN,70.0,,42.0,Goalkeeper,185.0,NaN,NaN


In [95]:
players_dict = players[['name','id']]
final_players_dict = {name: id for name, id in zip(players_dict['id'], players_dict['id'])}
final_clubs_dict = {name: id for name, id in zip(clubs_dict['id'], clubs_dict['id'])}
transfers['player_id'] = transfers['player_id'].map(final_players_dict).fillna('').replace('', None).astype(int)
transfers['origin_club_id'] = transfers['origin_club_id'].map(final_clubs_dict).fillna('').replace('', None).astype(int)
transfers['destination_club_id'] = transfers['destination_club_id'].map(final_clubs_dict).fillna('').replace('', None).astype(int)
transfers

,id,player_id,origin_club_id,destination_club_id,season,market_value,fee
0,NaN,207302,583,237,2022,4000000.0,NaN
1,NaN,316884,583,237,2021,2000000.0,2500000.0
2,NaN,316884,583,237,2022,300000.0,NaN
3,NaN,29051,583,237,2017,200000.0,NaN
4,NaN,277384,862,237,2019,4000000.0,1500000.0
...,...,...,...,...,...,...,...
22928,NaN,487688,418,16795,2020,3000000.0,2500000.0
22929,NaN,424784,1010,16795,2020,10000000.0,7000000.0
22930,NaN,610336,1084,16795,2020,1500000.0,1500000.0
22931,NaN,57825,3709,16795,2020,1500000.0,NaN


In [96]:
#players = players[players['current_club_id'].isin(final_clubs_dict.values())]
players['current_club_id'] = players['current_club_id'].map(final_clubs_dict).fillna('').replace('', None).astype(int)
players_season['player_id'] = players_season['player_id'].map(final_players_dict).fillna('').replace('', None).astype(int)
players = players.reset_index(drop=True)
players

,id,name,birth_date,age,height,current_club_id,club_joined,contract_expires,birth_place,citizenship,position,national_id,current_value,agent,foot,shirt_number
0,207302,Sergio Rico,1993-09-01 00:00:00,30.0,196.0,583,2022-01-21 00:00:00,None,None,None,Goalkeeper,None,4000000.0,None,right,None
1,316884,Dominik Greif,1997-04-06 00:00:00,26.0,197.0,237,2021-07-06 00:00:00,None,None,None,Goalkeeper,None,2000000.0,None,right,13.0
2,29051,Manolo Reina,1985-04-01 00:00:00,38.0,186.0,1084,2017-07-06 00:00:00,None,None,None,Goalkeeper,None,200000.0,None,right,None
3,632913,Pere García,2002-03-22 00:00:00,21.0,193.0,1084,None,None,None,None,Goalkeeper,None,None,None,right,None
4,741285,Ferran Quetglás,2005-06-06 00:00:00,18.0,188.0,1084,None,None,None,None,Goalkeeper,None,None,None,right,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8879,413459,Eris Abedini,1998-08-29 00:00:00,25.0,190.0,410,None,None,None,None,Defensive Midfield,None,300000.0,None,right,None
8880,284872,Álvaro Bravo,1998-02-04 00:00:00,25.0,181.0,410,None,None,None,None,Central Midfield,None,200000.0,None,right,None
8881,696883,Juan Brunet,1998-01-24 00:00:00,25.0,190.0,410,None,None,None,None,Central Midfield,None,50000.0,None,None,None
8882,636229,Carlos León,2002-12-05 00:00:00,21.0,185.0,410,None,None,None,None,Central Midfield,None,None,None,right,None


In [97]:
national.to_csv('national.csv', index=False)
competitions.to_csv('competitions.csv', index=False)
clubs.to_csv('clubs.csv', index=False)
players.to_csv('players.csv', index=False)
teams_season.to_csv('teams_season.csv', index=False)
players_season.to_csv('players_season.csv', index=False)
transfers.to_csv('transfers.csv', index=False)

In [98]:
writer = pd.ExcelWriter(r'E:\Quera\Projects\TransferMarket\Database\Quera_Data.xlsx')

national.to_excel(writer, sheet_name='national', index=False)
competitions.to_excel(writer, sheet_name='competitions', index=False)
clubs.to_excel(writer, sheet_name='clubs', index=False)
players.to_excel(writer, sheet_name='players', index=False)
teams_season.to_excel(writer, sheet_name='teams_season', index=False)
players_season.to_excel(writer, sheet_name='players_season', index=False)
transfers.to_excel(writer, sheet_name='transfers', index=False)

writer.save()